# Reference

YouTube - Leila Gharani - Convert Columns to Rows in Excel (NO Transpose, NO Formulas - SIMPLY UNPIVOT in Power Query)

https://youtu.be/JMrfbv2h7p8

## Example

In [1]:
import pandas as pd

xl_file = 'inputs/Power_Query_Transpose_Replace_XelPlus.xlsx'
df = pd.read_excel(xl_file, sheet_name='DataTable', skiprows=2).fillna(0)

# Ensure numeric values are correctly 'typed'
df.loc[:, 'Jan':'Dec'] = df.loc[:, 'Jan':'Dec'].astype(int)

# Set indexing
df = df.set_index(['App', 'Type'])
df

Jan  Feb  Mar  Apr  May   Jun   Jul   Aug   Sep   Oct  \
App       Type                                                             
WenCaL    Sales     22   32   44   54   68    76    89   110   113   191   
          Volume     0    0    0   54   68    76    89   110   113     0   
WenCal    Returns    0    0   22    0    0     0     0     0     0     0   
Blend     Sales    149  274  490  641  823  1003  1164  1272  1395  1538   
Voltage   Volume    47  144  268  358  430   516   608   710   796   866   
Inkly     Returns  119  139  245  350  410   499   547   520   550   551   
Right App Sales    146  307  433  560  669   780   884  1263  1263  1263   
Kind Ape  Volume    42   63  128  142  214   316   419   491   576   644   
Pet Feed  Returns   69  194  401  668  864  1083  1363  1364  1483  1720   

                    Nov   Dec  
App       Type                 
WenCaL    Sales     276   264  
          Volume      0     0  
WenCal    Returns     0     0  
Blend     Sales    1712  1750  
Voltage   Volume    990  1000  
Inkly     Returns   573   472  
Right App Sales    1265  1275  
Kind Ape  Volume    650   551  
Pet Feed  Returns  1909  1913

## Wide to long format

In [2]:
df_melt = df.reset_index().melt(id_vars=['App', 'Type'])
df_melt.head()

,App,Type,variable,value
0,WenCaL,Sales,Jan,22
1,WenCaL,Volume,Jan,0
2,WenCal,Returns,Jan,0
3,Blend,Sales,Jan,149
4,Voltage,Volume,Jan,47


## Long to wide format

In [3]:
from piper.defaults import *

piper v0.1.0: Monday, 29 March 2021 19:08:45


### Issue: months reported in wrong order

In [4]:
%%piper

df_melt 
>> pivot_table(index=['App', 'Type'], columns='variable', values='value')

Use %piper/%%piper --info to see rendered pandas pipe statement


name 'pivot_table' is not defined


### Fixed: monthly summary reporting (including totals)

Define months to categorical values

In [5]:
months = df_melt.variable.unique().tolist()
cat_months = pd.CategoricalDtype(categories=months, ordered=True)
set_category_type = lambda x: x.variable.astype(dtype=cat_months)

In [6]:
%%piper

df_melt 
# >> where("variable.isin(['Jan', 'Feb', 'Mar', 'Apr', 'May'])")
>> assign(variable=set_category_type)
>> pivot_table(index=['App', 'Type'], columns='variable', values='value')
>> adorn(axis='both', ignore_row_index=False)

Use %piper/%%piper --info to see rendered pandas pipe statement


name 'pivot_table' is not defined
